In [72]:
import requests
import json
import pandas as pd

import plotly.express as px

In [ ]:

argdict = {
    'state': 'NY',
    'climate_zone': '5A',
    'projection_case': 'MidCase',
    'design_areas': [
        {
            'type': 'ApartmentHighRise',
            'area': 200000,
            'heating_fuel': 'electricity',
            'dhw_fuel': 'Electricity',
            'heating_cop': 2.5,
            'dhw_cop': 2.5,
            'ashrae_standard': 'STD2013'
        },
        {
            'type': 'OfficeSmall',
            'area': 10000,
            'heating_fuel': 'Natural Gas',
            'dhw_fuel': 'Natural Gas',
            'heating_cop': 0.8,
            'dhw_cop': 0.8,
            'ashrae_standard': 'STD2013'
        }],
}

payload = json.dumps(argdict)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'


r = requests.get(
    f'{localhost}/get_projection_from_reference_buildings/?params={payload}'
)
js = r.json()

js

In [79]:
# manual enduses

config = {
    'state': 'MA',
    'projection_case': 'MidCase',
    'area': 215000,
    'enduses': [
        {
            'enduse': 'elec',
            'fuel': 'electricity',
            'kbtu_absolute': 7129e3
        },
        {
            'enduse': 'gas',
            'fuel': 'natural gas',
            'kbtu_absolute': 3111e3
        },
    ]
}

payload = json.dumps(config)
localhost = 'http://127.0.0.1:5000'
herokuhost =  'https://akf-becp-pyapi.herokuapp.com/'

r = requests.get(
    f'{localhost}/get_projection_from_manual_enduses/?params={payload}'
)

js = r.json()





In [80]:
df = pd.DataFrame(js['emissions_projection'])


px.line(df, x='year', y='kg_co2_per_sf')


